# Convolutional Neural Network

In [1]:
import math
import h5py
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [2]:
# open HDF5 file with all data
f = h5py.File('data.hdf5')

X = f['X_balanced'][...]
y = f['y_balanced'][...]

In [3]:
# constants as global variables
N_FEATURES = X.shape[1]
N_CLASSES = 5
LOG_DIR = '/data/podondra/tensorboard'

## One-hot Vector

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [4]:
# create one-hot vector 
enc = OneHotEncoder(dtype=np.int32, sparse=False)
y_one_hot = enc.fit_transform(y.reshape(-1, 1))
y_one_hot, y_one_hot.shape

(array([[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        ..., 
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=int32), (5755, 5))

In [5]:
sess = tf.InteractiveSession()

## The Model

In [6]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [7]:
def weight_variable(shape):
    '''Create weight variable. Initialize weights with
    small amount of noise to aviod dead neurons due to
    use of ReLU activation function.
    '''
    initial = tf.truncated_normal(shape, stddev=0.1)
    var = tf.Variable(initial, name='weight')
    variable_summaries(var)
    return var

def bias_variable(shape):
    '''Create bias variable. Initialize it to 0.1.'''
    initial = tf.constant(0.1, shape=shape)
    var = tf.Variable(initial, name='biases')
    variable_summaries(var)
    return var

https://www.tensorflow.org/versions/r0.11/api_docs/python/nn/convolution#conv2d
https://www.tensorflow.org/versions/r0.11/api_docs/python/nn/pooling#max_pool 

In [8]:
# convolution
def conv1d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# max pooling
def max_pool_1x2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 5, 1], strides=[1, 1, 5, 1], padding='SAME')

In [9]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, N_FEATURES], name='spectra')
    y_ = tf.placeholder(tf.float32, shape=[None, N_CLASSES], name='labels')

with tf.name_scope('input-reshape'):
    x_spectrum = tf.reshape(x, [-1, 1, N_FEATURES, 1])

# first convolutional layer
with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 1, 1, 32])
    b_conv1 = bias_variable([32])

    h_conv1 = tf.nn.relu(conv1d(x_spectrum, W_conv1) + b_conv1)
    h_pool1 = max_pool_1x2(h_conv1)

# second convolutional layer
with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 1, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv1d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_1x2(h_conv2)

# densely connected layer
with tf.name_scope('dense'):
    W_fc1 = weight_variable([160 * 64, 128])
    b_fc1 = bias_variable([128])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 160 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
with tf.name_scope('readout'):
    W_fc2 = weight_variable([128, N_CLASSES])
    b_fc2 = bias_variable([N_CLASSES])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

xentropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
)

tf.summary.scalar('xentropy', xentropy)

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(xentropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

merged = tf.summary.merge_all()

## TensorBoard

In [10]:
if tf.gfile.Exists(LOG_DIR):
    tf.gfile.DeleteRecursively(LOG_DIR)
tf.gfile.MakeDirs(LOG_DIR)
summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

## Cross-validation

In [ ]:
# stratified k-folds cross-validator
# the folds preserve the percentage for each class
skf = StratifiedKFold(n_splits=5)

# shuffle the matrixes
X, y = shuffle(X, y, random_state=0)

n_folds = 5
accuracies = np.zeros((n_folds,))

for iteration, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    # split to train and validation seet
    X_train, y_train = X[train_idx], y_one_hot[train_idx]
    X_valid, y_valid = X[valid_idx], y_one_hot[valid_idx]

    # preprocessing
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)

    sess.run(tf.global_variables_initializer())
    train_dict = {
        x: X_train_transformed,
        y_: y_train,
        keep_prob: 1.0
    }
    for i in range(1000):
        idx = np.random.choice(X_train_transformed.shape[0], 256, replace=False)
        batch_dict = {
            x: X_train_transformed[idx, :],
            y_: y_train[idx, :],
            keep_prob: 0.5
        }
        train_step.run(feed_dict=batch_dict)

        if i % 10 == 0:
            summary, train_accuracy, loss = sess.run(
                [merged, accuracy, xentropy],
                feed_dict={
                    x: X_train_transformed[idx, :],
                    y_: y_train[idx, :],
                    keep_prob: 1.0
                },
            )
            print('{:6} train {:6.6f} loss {:6.6f}'.format(i, train_accuracy, loss))
            writer.add_summary(summary, i)

    # evaluate
    X_valid_transformed = scaler.transform(X_valid)
    valid_dict = {
        x: X_valid_transformed,
        y_: y_valid,
        keep_prob: 1.0
    }
    valid_accuracy = accuracy.eval(feed_dict=valid_dict)
    accuracies[iteration] = valid_accuracy
    print('valid {}'.format(valid_accuracy))
    break

accuracies.mean(), accuracies.std()

In [ ]:
f.close()